In [1]:
from pycaret.regression import *
import pandas as pd

train_init = pd.read_csv('data/train.csv')
final_test = pd.read_csv('data/test.csv')
train_samp = train_init[['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'SalePrice']].sample(frac=0.10)
train_samp.shape

(146, 8)

In [2]:
setup(train_init, target='SalePrice', fold_shuffle=True, session_id=0)

,Description,Value
0,session_id,0
1,Target,SalePrice
2,Original Data,"(1460, 81)"
3,Missing Values,True
4,Numeric Features,20
5,Categorical Features,60
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1021, 390)"


(False,
 None,
 True,
 {'USI',
  'X',
  'X_test',
  'X_train',
  '_all_metrics',
  '_all_models',
  '_all_models_internal',
  '_available_plots',
  '_gpu_n_jobs_param',
  '_internal_pipeline',
  '_ml_usecase',
  'create_model_container',
  'dashboard_logger',
  'data_before_preprocess',
  'display_container',
  'exp_name_log',
  'experiment__',
  'fix_imbalance_method_param',
  'fix_imbalance_param',
  'fold_generator',
  'fold_groups_param',
  'fold_groups_param_full',
  'fold_param',
  'fold_shuffle_param',
  'gpu_param',
  'html_param',
  'imputation_classifier',
  'imputation_regressor',
  'iterative_imputation_iters_param',
  'log_plots_param',
  'logging_param',
  'master_model_container',
  'n_jobs_param',
  'prep_pipe',
  'pycaret_globals',
  'seed',
  'stratify_param',
  'target_param',
  'transform_target_method_param',
  'transform_target_param',
  'y',
  'y_test',
  'y_train'},
 Pipeline(memory=None, steps=[('empty_step', 'passthrough')], verbose=False),
 [],
 529     20062

In [34]:
best3 = compare_models(n_select=3, sort="RMSE", round=1)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,16789.5,815488014.5,26753.8,0.9,0.1,0.1,0.0
ridge,Ridge Regression,16395.3,802924961.2,26758.8,0.9,0.2,0.1,0.0
lasso,Lasso Regression,17613.9,881137167.2,28250.3,0.8,0.2,0.1,0.1
lr,Linear Regression,18263.6,915635232.2,28940.9,0.8,0.2,0.1,0.0
gbr,Gradient Boosting Regressor,17336.2,950265127.5,29140.8,0.8,0.1,0.1,0.3
lightgbm,Light Gradient Boosting Machine,17557.6,933452061.8,29148.9,0.9,0.1,0.1,0.1
rf,Random Forest Regressor,18368.3,983075461.9,30213.3,0.8,0.2,0.1,0.5
en,Elastic Net,20155.4,1088157257.7,31401.9,0.8,0.2,0.1,0.1
br,Bayesian Ridge,24013.5,1390196370.9,36093.8,0.8,0.2,0.1,0.1
ada,AdaBoost Regressor,24958.9,1417597503.2,36774.1,0.8,0.2,0.2,0.2


In [4]:
print(best3)

[Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=0, solver='auto', tol=0.001), OrthogonalMatchingPursuit(fit_intercept=True, n_nonzero_coefs=None,
                          normalize=True, precompute='auto', tol=None), LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)]


In [5]:
ridge = create_model('ridge', round=1)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,14916.1,551933203.7,23493.3,0.9,0.1,0.1
1,16770.9,900707659.5,30011.8,0.9,0.1,0.1
2,16694.9,495117478.1,22251.2,0.9,0.2,0.1
3,12958.5,311568468.8,17651.3,0.9,0.1,0.1
4,15156.1,408222185.8,20204.5,0.9,0.1,0.1
5,16152.8,538184729.4,23198.8,0.9,0.4,0.1
6,16601.8,570171737.7,23878.3,0.9,0.1,0.1
7,17115.4,601842943.3,24532.5,0.9,0.2,0.1
8,20092.4,2763766192.0,52571.5,0.5,0.2,0.1


In [16]:
ridge_tuned = tune_model(ridge, n_iter = 50)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,13706.7864,501317954.5037,22390.1307,0.8959,0.1208,0.0813
1,18357.6648,1160889907.7346,34071.8345,0.8867,0.1347,0.0929
2,14929.0754,424622189.6440,20606.3628,0.9167,0.1323,0.0915
3,13412.9743,334023952.2417,18276.3222,0.9198,0.1199,0.0860
4,13684.5392,325238998.6505,18034.3838,0.9345,0.1294,0.0865
5,14976.1452,454669358.7885,21322.9772,0.9273,0.1151,0.0844
6,15923.3339,518714884.1736,22775.3130,0.8950,0.1299,0.0978
7,17218.5803,567277875.4152,23817.5959,0.9008,0.1497,0.1125
8,20361.9596,2822530403.1190,53127.4920,0.4903,0.1983,0.1307


In [13]:
best3_tuned = [tune_model(model) for model in best3]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,14885.1709,624097072.3770,24981.9349,0.8704,0.1219,0.0847
1,24267.2198,1948190690.0655,44138.3132,0.8099,0.1700,0.1203
2,14908.5582,466247671.9741,21592.7690,0.9086,0.1544,0.1007
3,15409.1385,628054513.6285,25061.0158,0.8492,0.1416,0.0943
4,15415.7253,521590766.5944,22838.3617,0.8950,0.1245,0.0875
5,15546.9424,587866515.3374,24245.9587,0.9060,0.1201,0.0892
6,16456.7554,571354511.6276,23903.0231,0.8843,0.1395,0.1017
7,17714.6009,629674825.4086,25093.3223,0.8899,0.1650,0.1221
8,20634.1235,1911870130.2979,43724.9372,0.6548,0.1790,0.1231


In [14]:
print(best3_tuned)

[Ridge(alpha=4.14, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=0, solver='auto', tol=0.001), OrthogonalMatchingPursuit(fit_intercept=True, n_nonzero_coefs=139,
                          normalize=False, precompute='auto', tol=None), LGBMRegressor(bagging_fraction=0.5, bagging_freq=0, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=1.0,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=11, min_child_weight=0.001, min_split_gain=0.5,
              n_estimators=80, n_jobs=-1, num_leaves=50, objective=None,
              random_state=0, reg_alpha=0.01, reg_lambda=0.005, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)]


In [ ]:

plot_model(ridge_tuned)

In [27]:

ridge_bagged = ensemble_model(ridge_tuned)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,13175.7744,465257962.6287,21569.8392,0.9034,0.1107,0.0774
1,18089.6278,1086539461.7735,32962.6980,0.8940,0.1298,0.0910
2,14658.8659,421253881.0955,20524.4703,0.9174,0.1308,0.0903
3,13663.2836,339266917.7870,18419.1997,0.9185,0.1222,0.0879
4,13958.7972,306513645.5584,17507.5311,0.9383,0.1183,0.0862
5,14518.0241,409757741.7910,20242.4737,0.9345,0.1075,0.0826
6,16274.1645,539425267.8005,23225.5305,0.8908,0.1293,0.0983
7,17599.6931,575570802.0409,23991.0567,0.8994,0.1491,0.1162
8,21266.3207,2921773720.1839,54053.4339,0.4724,0.2041,0.1357


In [28]:
ridge_boosted = ensemble_model(ridge_tuned, method='Boosting')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,21423.0596,1086298441.8921,32959.0419,0.7744,0.2798,0.1312
1,22945.1391,1454799645.6471,38141.8359,0.8581,0.1751,0.1250
2,21605.6286,789410809.9163,28096.4555,0.8452,0.1861,0.1299
3,17220.9614,509435931.8875,22570.6874,0.8777,0.1519,0.1109
4,19103.9884,633432971.3265,25168.0943,0.8725,0.1740,0.1204
5,21395.9499,900018232.7378,30000.3039,0.8561,0.3828,0.1245
6,20358.0951,809557897.2203,28452.7309,0.8361,0.1578,0.1240
7,22748.4173,981264240.1355,31325.1375,0.8284,0.2307,0.1484
8,27518.1385,3576590825.0550,59804.6054,0.3541,0.2549,0.1809


In [41]:
omp = create_model('omp', fold=20)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10644.5752,248845272.8712,15774.8304,0.9409,0.0910,0.0621
1,17127.7138,779323857.9057,27916.3726,0.8566,0.1451,0.1010
2,17740.0327,742955764.9786,27257.2149,0.9285,0.1255,0.0974
3,21585.8191,1384613195.2643,37210.3910,0.8622,0.1819,0.1244
4,15555.9385,416783031.1898,20415.2647,0.9296,0.1377,0.0965
5,14294.8893,463000830.3874,21517.4541,0.8911,0.1566,0.1020
6,12989.8397,303412143.3081,17418.7297,0.9232,0.1279,0.0895
7,13470.8262,333581604.2019,18264.2165,0.9232,0.1093,0.0814
8,13363.2200,294521465.4742,17161.6277,0.9488,0.1290,0.0800


In [42]:

omp_tuned = tune_model(omp, n_iter=100)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,13617.3076,507536452.9145,22528.5697,0.8946,0.1199,0.0814
1,16030.1531,679129399.8963,26060.1113,0.9337,0.1225,0.0884
2,14741.2767,426506315.8911,20652.0293,0.9164,0.1475,0.0955
3,12787.2655,310830454.5831,17630.3844,0.9254,0.1253,0.0852
4,14995.0766,351909754.3300,18759.2578,0.9292,0.1215,0.0913
5,14852.9358,434404883.2022,20842.3819,0.9305,0.1066,0.0846
6,16508.6753,552853773.3685,23512.8427,0.8881,0.1282,0.0960
7,16883.3528,621080096.6749,24921.4786,0.8914,0.1519,0.1108
8,21120.5738,2737731138.5281,52323.3326,0.5056,0.1931,0.1311


In [43]:
omp_bagged = ensemble_model(omp_tuned)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,13937.1682,499991804.4494,22360.4965,0.8962,0.1148,0.0818
1,17403.7965,839683458.1729,28977.2921,0.9181,0.1288,0.0928
2,15460.8234,456101605.4665,21356.5354,0.9106,0.1497,0.1002
3,13244.7267,320496129.6161,17902.4057,0.9230,0.1258,0.0873
4,14611.7596,326874356.7939,18079.6669,0.9342,0.1145,0.0889
5,14700.3650,384582763.9381,19610.7818,0.9385,0.1107,0.0873
6,16998.2755,600657999.6030,24508.3251,0.8784,0.1474,0.1047
7,16310.2555,559060424.8570,23644.4587,0.9022,0.1427,0.1087
8,20962.2635,2859407182.9518,53473.4250,0.4836,0.2010,0.1342


In [38]:
lasso = create_model('lasso', fold=20)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,13805.7608,378176437.7320,19446.7591,0.9101,0.1703,0.0849
1,17635.8333,910516162.1567,30174.7603,0.8324,0.1783,0.1086
2,15308.9853,493641108.3225,22218.0357,0.9525,0.1057,0.0842
3,19176.3603,1030720614.9280,32104.8379,0.8974,0.1671,0.1129
4,20200.8162,846586099.8280,29096.1527,0.8570,0.2252,0.1304
5,15041.7206,474730537.0453,21788.3119,0.8883,0.1524,0.0975
6,12415.4314,296640702.0979,17223.2605,0.9249,0.1603,0.0859
7,16561.4314,532369117.3427,23073.1254,0.8774,0.1355,0.1010
8,14773.2549,313491689.6964,17705.6965,0.9455,0.1105,0.0865


In [39]:
lasso_tuned = tune_model(lasso, n_iter=100)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,13035.9733,462334292.8954,21501.9602,0.9040,0.1134,0.0764
1,15352.5588,728710623.0495,26994.6406,0.9289,0.1212,0.0813
2,14435.8168,392579246.5735,19813.6127,0.9230,0.1437,0.0943
3,12124.5012,315069956.3765,17750.2100,0.9243,0.1008,0.0727
4,13883.0944,365428400.3320,19116.1816,0.9264,0.1083,0.0823
5,14565.0221,445725351.3105,21112.2086,0.9287,0.0994,0.0784
6,14876.9657,499147136.1288,22341.6010,0.8989,0.1115,0.0838
7,15609.1900,545840737.8350,23363.2347,0.9046,0.1407,0.1013
8,19313.9920,2724094379.8429,52192.8576,0.5081,0.1818,0.1175


In [40]:
lasso_bagged = ensemble_model(lasso_tuned)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,13185.2426,459931561.2213,21446.0150,0.9045,0.1130,0.0777
1,16044.4510,815933356.9365,28564.5472,0.9204,0.1215,0.0821
2,14494.3935,399423768.2007,19985.5890,0.9217,0.1391,0.0940
3,11858.4185,296647849.0927,17223.4680,0.9288,0.0990,0.0722
4,13847.0051,312850344.7475,17687.5760,0.9370,0.1061,0.0820
5,14277.6318,428917339.3591,20710.3196,0.9314,0.0999,0.0788
6,15819.9921,542266068.2225,23286.6071,0.8902,0.1155,0.0893
7,15358.7681,537955849.3957,23193.8753,0.9059,0.1377,0.0994
8,19184.0018,2628551864.2158,51269.4048,0.5253,0.1790,0.1169


In [11]:
blender_top3 = blend_models(best3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,12363.3368,446045919.9686,21119.7992,0.9074,0.1072,0.0726
1,17263.1016,1006585328.9320,31726.7289,0.9018,0.1311,0.0901
2,13019.2319,327880153.1867,18107.4613,0.9357,0.1283,0.0848
3,11661.1081,293753035.4279,17139.2251,0.9295,0.0996,0.0705
4,12776.0279,298459634.2226,17275.9843,0.9399,0.1095,0.0780
5,13994.5893,409089199.0802,20225.9536,0.9346,0.0967,0.0784
6,15258.8352,505039787.3935,22473.0903,0.8977,0.1260,0.0919
7,15256.8001,467411145.0308,21619.6935,0.9183,0.1308,0.0990
8,19239.1082,2490847146.2674,49908.3875,0.5502,0.1817,0.1178


In [25]:
blender_best3_tuned = blend_models(best3_tuned)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,12166.6655,452194764.8385,21264.8716,0.9061,0.1062,0.0717
1,17921.4419,1061883272.0563,32586.5505,0.8964,0.1294,0.0898
2,13363.9800,343284355.1224,18527.9345,0.9327,0.1289,0.0866
3,12326.2346,334468595.1254,18288.4826,0.9197,0.1121,0.0764
4,13041.9362,290865031.0205,17054.7656,0.9415,0.1174,0.0820
5,13204.7921,394397416.8152,19859.4415,0.9369,0.0922,0.0729
6,14188.5009,454947981.9552,21329.5097,0.9079,0.1110,0.0834
7,15153.7934,444114784.0280,21074.0310,0.9223,0.1308,0.0995
8,18910.5956,2367992888.7801,48662.0272,0.5724,0.1801,0.1165


In [26]:
stacked_best3 = stack_models(best3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,15544.9106,664242357.4749,25772.8997,0.8621,0.1493,0.0967
1,18478.0710,938335585.2090,30632.2638,0.9085,0.1473,0.1016
2,17000.3800,618813432.7325,24875.9609,0.8787,0.1796,0.1056
3,15368.0588,466005904.6118,21587.1699,0.8881,0.1524,0.1027
4,17916.2652,563101379.1223,23729.7572,0.8867,0.1691,0.1100
5,17924.8109,609778319.1084,24693.6899,0.9025,0.1450,0.1043
6,17693.1987,651704811.7814,25528.5098,0.8680,0.2167,0.1093
7,17038.2076,610206842.0237,24702.3651,0.8933,0.2216,0.1123
8,21564.1035,2633618053.5812,51318.7885,0.5244,0.1913,0.1322


In [44]:
blend_best3_tuned_twice = blend_models([ridge_bagged, omp_tuned, lasso_bagged])


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,12927.4560,451231153.5165,21242.2022,0.9063,0.1088,0.0755
1,16433.6415,813903054.4609,28528.9862,0.9206,0.1207,0.0854
2,14302.5487,395126894.8651,19877.7990,0.9225,0.1348,0.0914
3,12389.0147,281196572.4651,16768.9169,0.9325,0.1099,0.0796
4,13745.2145,296491084.8202,17218.9165,0.9403,0.1105,0.0841
5,14142.1674,407466539.5503,20185.8004,0.9349,0.1002,0.0789
6,15694.5889,521471502.1975,22835.7505,0.8944,0.1155,0.0890
7,16223.9275,553607074.8878,23528.8562,0.9032,0.1423,0.1066
8,20042.8698,2691248849.2983,51877.2479,0.5140,0.1887,0.1249


In [45]:
final_model = finalize_model(blend_best3_tuned_twice)


In [47]:
pred = predict_model(final_model, final_test)
pred


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Label
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,130557.569473
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,167347.365390
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,194015.769389
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,203324.316475
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,219952.467114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,90987.896903
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,86148.403564
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,173931.161850
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,127907.290438


In [50]:

res = pd.DataFrame({
        "Id": pred['Id'],
        "SalePrice": pred['Label']
    })

res.to_csv('submissions/submission.csv', index=False)